In [41]:
import pandas as pd
import numpy as np

In [62]:
# data_file
data = pd.read_excel(r"D:\Documents\UIUC\ICSO\RommateMatch\File\MatchFile.xlsx")
# weight
arbitary_large_num = 10000000
party = 4
music = 3
keyboard = 2
clean = 3

In [65]:
# Preprocess data, replace some cells and drop duplicate emails/names
data_copy = data.copy().drop(columns=["提交者（自动）","提交时间（自动）"]).drop_duplicates(['您的学校邮箱（必填）']).drop_duplicates(['姓名（必填）'])
data_copy = data_copy.replace({'Male':0,"Female":1,False:1, True:0, 'Yes':1, 'No':0})
data_copy = data_copy.replace(['Before 22:00', '22:00 - 22:59', "23:00 - 23:59", "00:00 - 00:59", 'After 01:00 a.m.'], [1, 2, 3, 4, 5])
data_copy = data_copy.replace(["Before 06:00", "06:00 - 06:59", "07:00 - 07:59", "08:00 - 08:59", "After 09:00 a.m."], [1, 2, 3, 4, 5])
data_copy.head()

,姓名（必填）,性别（必填）,您的学校邮箱（必填）,所偏好的所有PCH宿舍（必填）,所偏好的所有URH宿舍（必填）,是否有在寝室开party的需求（必填）,是否有在寝室玩乐器的需求（必填）,是否有使用机械键盘的习惯（必填）,平均睡觉时间（必填）,平均起床时间（必填）,...,室友和您睡觉时间接近的重要程度（必填）,希望室友所在的学院（可多选）（必填）,我同意将我的邮箱自动发送给推荐的匹配室友（必填）,我已知晓除姓名邮箱以外的所有信息都不会泄漏给任何个体或者组织（必填）,是否有饮酒习惯（必填）,是否有吸烟习惯,室友保持整洁干净的重要性（必填）,是否接受室友吸烟（必填）,是否接受室友饮酒（必填）,希望留给推荐匹配室友的话（必填）
0,战术核显卡,0,chenhui5@illinois.edu,Presby Hall,URBANA NORTH (Unit One/Allen Hall）,1,1,1,5,5,...,3,"The Grainger College of Engineering., College ...",0,0,1,1,5,0,0,Lol
1,嘤嘤嘤呜呜呜呜嘤嘤嘤,0,yihongj2@illinois.edu,"Illini Tower, Newman Hall, Presby Hall, None o...","URBANA NORTH (Unit One/Allen Hall）, URBANA NOR...",0,1,0,2,4,...,5,"The Grainger College of Engineering., College ...",0,0,0,1,3,0,1,嘤嘤嘤呜呜呜嘤嘤嘤
2,孙山,0,shansun2@illinois.edu,None of above,IKENBERRY COMMONS SOUTH (Snyder),0,1,0,3,3,...,0,"The Grainger College of Engineering., College ...",0,0,0,1,3,0,1,你好 我是傻狗
3,abc,0,yj17@illinois.edu,"Armory House, Bromley Hall, Brown House on Col...","URBANA NORTH (Unit One/Allen Hall）, URBANA NOR...",1,1,1,3,3,...,3,"The Grainger College of Engineering., College ...",0,0,1,1,4,0,0,hhhh


In [68]:
# College and desire dorm hasn't been considered.
for j in range(0, 4):
    own_data = np.array([
        data_copy.at[j, '性别（必填）'], 
        data_copy.at[j, '是否有在寝室开party的需求（必填）'], 
        data_copy.at[j, '是否有在寝室玩乐器的需求（必填）'], 
        data_copy.at[j, '是否有使用机械键盘的习惯（必填）'], 
        data_copy.at[j, '平均睡觉时间（必填）'], 
        data_copy.at[j, '平均起床时间（必填）'], 
        data_copy.at[j, '是否有饮酒习惯（必填）'], 
        data_copy.at[j, '是否有吸烟习惯'],
        data_copy.at[j, '室友保持整洁干净的重要性（必填）']
    ])
    # Weight sequence: gender, party, instrument usage, mechanical keyboard, bed time, drink, smoke and cleaness
    weight = np.array([
        arbitary_large_num,
        party,
        music,
        keyboard,
        data_copy.at[j, '室友和您睡觉时间接近的重要程度（必填）'],
        data_copy.at[j, '室友和您睡觉时间接近的重要程度（必填）'],
        arbitary_large_num,
        arbitary_large_num,
        clean
    ])
    for i in range(0, 4):
        i_data = np.array([
            data_copy.at[i, '性别（必填）'], 
            data_copy.at[i, '是否有在寝室开party的需求（必填）'], 
            data_copy.at[i, '是否有在寝室玩乐器的需求（必填）'], 
            data_copy.at[i, '是否有使用机械键盘的习惯（必填）'], 
            data_copy.at[i, '平均睡觉时间（必填）'], 
            data_copy.at[i, '平均起床时间（必填）'], 
            data_copy.at[i, '是否有饮酒习惯（必填）'], 
            data_copy.at[i, '是否有吸烟习惯'],
            data_copy.at[i, '室友保持整洁干净的重要性（必填）']
        ])
        score = np.linalg.norm((own_data-i_data) * (weight), 2)
        print('%s\'s score with with %s is %f'%(data_copy.at[j, "姓名（必填）"], data_copy.at[i, "姓名（必填）"], score))

战术核显卡's score with with 战术核显卡 is 0.000000
战术核显卡's score with with 嘤嘤嘤呜呜呜呜嘤嘤嘤 is 10000000.000007
战术核显卡's score with with 孙山 is 10000000.000006
战术核显卡's score with with abc is 9.000000
嘤嘤嘤呜呜呜呜嘤嘤嘤's score with with 战术核显卡 is 10000000.000015
嘤嘤嘤呜呜呜呜嘤嘤嘤's score with with 嘤嘤嘤呜呜呜呜嘤嘤嘤 is 0.000000
嘤嘤嘤呜呜呜呜嘤嘤嘤's score with with 孙山 is 7.071068
嘤嘤嘤呜呜呜呜嘤嘤嘤's score with with abc is 10000000.000004
孙山's score with with 战术核显卡 is 10000000.000003
孙山's score with with 嘤嘤嘤呜呜呜呜嘤嘤嘤 is 0.000000
孙山's score with with 孙山 is 0.000000
孙山's score with with abc is 10000000.000001
abc's score with with 战术核显卡 is 9.000000
abc's score with with 嘤嘤嘤呜呜呜呜嘤嘤嘤 is 10000000.000002
abc's score with with 孙山 is 10000000.000001
abc's score with with abc is 0.000000
